<a href="https://colab.research.google.com/github/shasso2s/pytorch/blob/main/Scaling_technik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import mnist
# example of using ImageDataGenerator to normalize images
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.preprocessing.image import ImageDataGenerator

**Normalization using ImageDataGenerator**
*italicized text*
In the following code we are using Pixel Normalization 
  * **Traget**: Normalization of Pixel values and transform it into range[0-1]
Normaly the Pixel values are always between [0-255] , those values would be very high for the machine in case of procesing, therfore we use rescale(1/255) to put all the pixel value in this range.

* Before we aply the scale we are reshaping all the images in one Size und Width(28*28) and channel =1 # RGB.
* After that we are transforming the Label into categorical[0-10]

* Imagedatagenaration is  a such a beatiful class that wrap the dataset , and make it in Batch with a specific size using **flow function**

* **The ImageDataGenerator does not need to be fit in this case because there are no global statistics that need to be calculated.** therfore and in comparaison to other technique we don't use fit in scaling.

* After scaling the pixel value and put dataset in form of Batch comes the step of model building , compiling, evaluation.

In [ ]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
print('train',x_train.shape)
print('test',x_test.shape)
print('pixel value in x_train ',x_train.min(),x_train.max(),x_train.std(),x_train.mean())
print('pixel value in x_test',x_test.min(),x_test.max(),x_test.std(),x_test.mean())
width,height,channel=x_train.shape[1],x_train.shape[2],1
x_train=x_train.reshape(x_train.shape[0],width,height,channel)
x_test=x_test.reshape(x_test.shape[0],width,height,channel)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
print("train min=%.3f, max=%.3f" %(x_train.min(),x_train.max()))
print("test min=%.3f, test max=%.3f"% (x_test.min(),x_test.max()))
datagen=ImageDataGenerator(rescale=1.0/255.0)
train_iterator=datagen.flow(x_train,y_train,batch_size=64)
test_iterator=datagen.flow(x_test,y_test,batch_size=64)
print("batches train , test", len(train_iterator),len(test_iterator))
batchx,batchy=train_iterator.next()
print('batchx shape=%s , batchx_min=%.3f, batchx_max=%.3f'% (batchx.shape,batchx.min(), batchx.max()))
import tensorflow as tf
model=tf.keras.Sequential([
                           tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(width,height,channel)),
                           tf.keras.layers.MaxPooling2D(2,2),
                           tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                           tf.keras.layers.MaxPooling2D(2,2),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(64,activation='relu'),
                           tf.keras.layers.Dense(10,activation='softmax')])
# compile the model:
model.compile(optimizer='adam',loss=tf.keras.metrics.categorical_crossentropy,metrics=['accuracy'])
# fit the model:
model.fit_generator(train_iterator,steps_per_epoch =len(train_iterator),epochs=5)
#evalaute the model:
_,acc=model.evaluate_generator(test_iterator,steps=len(test_iterator),verbose=0)
print('Test accuracy:%.3f'%(acc*100))

train (60000, 28, 28)
test (10000, 28, 28)
pixel value in x_train  0 255 78.56748998339798 33.318421449829934
pixel value in x_test 0 255 79.17246322228644 33.791224489795916
train min=0.000, max=255.000
test min=0.000, test max=255.000
batches train , test 938 157
batchx shape=(64, 28, 28, 1) , batchx_min=0.000, batchx_max=1.000
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


938/938 [==============================] - 51s 54ms/step - loss: 0.1642 - accuracy: 0.9514
Epoch 2/5
938/938 [==============================] - 50s 54ms/step - loss: 0.0536 - accuracy: 0.9837
Epoch 3/5
938/938 [==============================] - 50s 54ms/step - loss: 0.0373 - accuracy: 0.9885
Epoch 4/5
938/938 [==============================] - 51s 54ms/step - loss: 0.0293 - accuracy: 0.9908
Epoch 5/5
938/938 [==============================] - 50s 54ms/step - loss: 0.0223 - accuracy: 0.9928


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


Test accuracy:98.910


**Pixel Centering**: scale pixel values to have a zero mean.
* Another popular pixel scaling method is to calculate the mean pixel value across the entire training dataset, then subtract it from each image.

* This is called centering and has the effect of centering the distribution of pixel values on zero: that is, the mean pixel value for centered images will be zero.
* **datagen = ImageDataGenerator(featurewise_center=True)** this code alow us to achieve pixel centering , after that comes fit, anf flow function 
* the next steps is exactly the same as Scaling


In [ ]:
(trainX, trainY), (testX, testY) = mnist.load_data()
# reshape dataset to have a single channel
width, height, channels = trainX.shape[1], trainX.shape[2], 1
trainX = trainX.reshape((trainX.shape[0], width, height, channels))
testX = testX.reshape((testX.shape[0], width, height, channels))
# one hot encode target values
trainY = to_categorical(trainY)
testY = to_categorical(testY)
# create generator to center images
datagen = ImageDataGenerator(featurewise_center=True)
# calculate mean on training dataset
datagen.fit(trainX)
# prepare an iterators to scale images
train_iterator = datagen.flow(trainX, trainY, batch_size=64)
test_iterator = datagen.flow(testX, testY, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator), len(test_iterator)))
# define model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))
# compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# fit model with generator
model.fit_generator(train_iterator, steps_per_epoch=len(train_iterator), epochs=5)
# evaluate model
_, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Batches train=938, test=157
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


938/938 [==============================] - 49s 52ms/step - loss: 0.4301 - accuracy: 0.9255
Epoch 2/5
938/938 [==============================] - 48s 51ms/step - loss: 0.0678 - accuracy: 0.9799
Epoch 3/5
938/938 [==============================] - 47s 50ms/step - loss: 0.0487 - accuracy: 0.9850
Epoch 4/5
938/938 [==============================] - 52s 55ms/step - loss: 0.0389 - accuracy: 0.9882
Epoch 5/5
938/938 [==============================] - 50s 53ms/step - loss: 0.0379 - accuracy: 0.9880


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.


Test Accuracy: 98.570


In [ ]:
t

**How to Standardize Image with ImageDataGenerator **
Standardization of images is achieved by substracting the mean pixel value and dividing the result by the standard deviation of the pixel values
* **datagen=ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)**



In [ ]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
width,height,channel=x_train.shape[1],x_train.shape[2],1
x_train=x_train.reshape(x_train.shape[0],width,height,channel)
x_test=x_test.reshape(x_test.shape[0],width,height,channel)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
datagen=ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True)
datagen.fit(x_train)
x_train_iterator=datagen.flow(x_train,y_train,batch_size=64)
y_test_iterator=datagen.flow(x_test,y_test,batch_size=64)
batchx,batchy=x_train_iterator.next()
# model
model=tf.keras.Sequential([
                           tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(width,height,channel)),
                           tf.keras.layers.MaxPooling2D(2,2),
                           tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                           tf.keras.layers.MaxPooling2D(2,2),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(64,activation='relu'),
                           tf.keras.layers.Dense(10,activation='softmax')])
#compile the model:
model.compile(optimizer='adam',loss=tf.keras.metrics.categorical_crossentropy,metrics=['accuracy'])                        

#fit the model:
model.fit_generator(x_train_iterator,steps_per_epoch=len(x_train_iterator),epochs=5)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
938/938 [==============================] - 50s 53ms/step - loss: 0.1392 - accuracy: 0.9590
Epoch 2/5
938/938 [==============================] - 48s 52ms/step - loss: 0.0430 - accuracy: 0.9870
Epoch 3/5
938/938 [==============================] - 48s 51ms/step - loss: 0.0297 - accuracy: 0.9907
Epoch 4/5
938/938 [==============================] - 49s 52ms/step - loss: 0.0237 - accuracy: 0.9927
Epoch 5/5
938/938 [==============================] - 49s 52ms/step - loss: 0.0179 - accuracy: 0.9941


In [ ]:
 _,acc=model.evaluate_generator(y_test_iterator,steps=len(y_test_iterator),verbose=0)
print('acuracy:%.3f' %(acc*100))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


acuracy:99.080
